# **Importing Libraries**

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# **Functions Definitions**

In [0]:
def gaussian_function(x,m,s):
  x = np.array(x);
  m = np.array(m);
  s = np.array(s);
  k = m.size;
  factor = 1/np.sqrt(((2*np.pi)**k)*np.linalg.det(s));
  ret_arr = []
  for i in range(x.shape[0]):
    exponent = (-0.5) * np.matmul(np.matmul((x[i]-m),np.linalg.inv(s)),(x[i]-m).T);
    ret_arr.append(np.exp(exponent))

  return factor*np.array(ret_arr)

def naive_bayes_classifier(x,name=None):
  x = np.array(x)
  z = []
  if(name == None):
    for i in range(x.shape[0]):
      z.append(np.argmax(x[i])+1)
  else:
    for i in range(x.shape[0]):
      z.append(name[np.argmax(x[i])])
  return z

def loss_function(x,y):
  x = np.array(x)
  y = np.array(y)
  if(x.size == y.size):
    L = np.zeros(x.size)
    for i in range(x.size):
      if(x[i] == y[i]):
        L[i] = 0;
      else:
        L[i] = 1;
    return L;
  else:
    print("Input arrays with same dimensions")   
    return 
  
def MLE_BDT(P,p): # P = [0.7,0.3], p = [1/3,1/3,1/3]
  print("Training dataset size = {}% and testing dataset size = {}%".format(P[0]*100,P[1]*100))
  print("Prior distribution = {}".format(p))

  df = pd.read_csv("irisdata.csv", sep=',')

  attributes = ["sepal_length", "sepal_width", "petal_length", "petal_width", "Class"]
  df.columns = attributes
  #train_df = df.sample(frac=P[0])
  #test_df = df.sample(frac=P[1])

  classes = df['Class'].unique().tolist()
  for cls, df_cls in df.groupby('Class'):
    if(cls == classes[0]):
      df_0 = df_cls
    elif(cls == classes[1]):
      df_1 = df_cls
    elif(cls == classes[2]):
      df_2 = df_cls

  
  train_df_0, test_df_0 = train_test_split(df_0, test_size=P[1])
  train_df_1, test_df_1 = train_test_split(df_1, test_size=P[1])
  train_df_2, test_df_2 = train_test_split(df_2, test_size=P[1])

  train_df = pd.concat([train_df_0, train_df_1, train_df_2], ignore_index=True)
  test_df = pd.concat([test_df_0, test_df_1, test_df_2], ignore_index=True)
  
  m = np.array([train_df_0.drop(["Class"],axis=1).mean().to_numpy(),\
    train_df_1.drop(["Class"],axis=1).mean().to_numpy(),\
    train_df_2.drop(["Class"],axis=1).mean().to_numpy()])

  print(m)
  s = np.array([train_df_0.drop(["Class"],axis=1).cov().to_numpy(),\
    train_df_1.drop(["Class"],axis=1).cov().to_numpy(),\
    train_df_2.drop(["Class"],axis=1).cov().to_numpy()])

  p_x_0 = gaussian_function(train_df.drop(["Class"],axis=1),m[0],s[0])
  p_x_1 = gaussian_function(train_df.drop(["Class"],axis=1),m[1],s[1])
  p_x_2 = gaussian_function(train_df.drop(["Class"],axis=1),m[2],s[2])
  train_df["P(X|0)"] = p_x_0;
  train_df["P(X|1)"] = p_x_1;
  train_df["P(X|2)"] = p_x_2;

  lp_x_0 = np.log(p_x_0)
  lp_x_1 = np.log(p_x_1)
  lp_x_2 = np.log(p_x_2)
  print(np.sum(lp_x_0),np.sum(lp_x_1),np.sum(lp_x_2))
  p_0_x = (p_x_0*p[0])/(p_x_0*p[0] + p_x_1*p[1] + p_x_2*p[2])
  p_1_x = (p_x_1*p[1])/(p_x_0*p[0] + p_x_1*p[1] + p_x_2*p[2])
  p_2_x = (p_x_2*p[2])/(p_x_0*p[0] + p_x_1*p[1] + p_x_2*p[2])
  train_df["P(0|X)"] = p_0_x;
  train_df["P(1|X)"] = p_1_x;
  train_df["P(2|X)"] = p_2_x;

  train_df["Prediction"] = naive_bayes_classifier(\
    np.column_stack((p_0_x,p_1_x,p_2_x)),classes)

  train_df["Loss"] = loss_function(\
    train_df["Prediction"].to_numpy(),train_df["Class"].to_numpy())

  for loss, df_loss in train_df.groupby('Loss'):
    if(loss == 0):
      loss_df_0 = df_loss
    elif(loss == 1):
      loss_df_1 = df_loss

  print("Accuracy of model on training set is: {}%".format((1-train_df["Loss"].mean())*100))

  p_x_0 = gaussian_function(test_df.drop(["Class"],axis=1),m[0],s[0])
  p_x_1 = gaussian_function(test_df.drop(["Class"],axis=1),m[1],s[1])
  p_x_2 = gaussian_function(test_df.drop(["Class"],axis=1),m[2],s[2])

  test_df["P(X|0)"] = p_x_0;
  test_df["P(X|1)"] = p_x_1;
  test_df["P(X|2)"] = p_x_2;
  
  p_0_x = (p_x_0*p[0])/(p_x_0*p[0] + p_x_1*p[1] + p_x_2*p[2])
  p_1_x = (p_x_1*p[1])/(p_x_0*p[0] + p_x_1*p[1] + p_x_2*p[2])
  p_2_x = (p_x_2*p[2])/(p_x_0*p[0] + p_x_1*p[1] + p_x_2*p[2])
  test_df["P(0|X)"] = p_0_x;
  test_df["P(1|X)"] = p_1_x;
  test_df["P(2|X)"] = p_2_x;

  test_df["Prediction"] = naive_bayes_classifier(np.column_stack((p_0_x,p_1_x,p_2_x)),classes)

  test_df["Loss"] = loss_function(test_df["Prediction"].to_numpy(),test_df["Class"].to_numpy())

  print("Accuracy of model on test set is: {}% \n".format((1-test_df["Loss"].mean())*100))

# **Model Training and Evaluation**

In [0]:
MLE_BDT([0.7,0.3],[1/3,1/3,1/3])

Training dataset size = 70.0% and testing dataset size = 30.0%
Prior distribution = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
[[5.04285714 3.5        1.47142857 0.24857143]
 [5.92571429 2.78       4.27142857 1.34285714]
 [6.58285714 3.         5.51142857 2.00571429]]
-16854.93899231041 -2621.407256684148 -4507.260075848843
Accuracy of model on training set is: 98.09523809523809%
Accuracy of model on test set is: 100.0% 



In [0]:
P = np.array([[0.7,0.3],[0.5,0.5],[0.3,0.7]])
p = np.array([[1/3,1/3,1/3],[0.3,0.5,0.2]])
for i in range(P.shape[0]):
  for j in range(p.shape[0]):
    MLE_BDT(P[i],p[j])


Training dataset size = 70.0% and testing dataset size = 30.0%
Prior distribution = [0.33333333 0.33333333 0.33333333]
[[5.06857143 3.44857143 1.47714286 0.25714286]
 [5.87428571 2.77714286 4.19428571 1.32285714]
 [6.54       2.97714286 5.52       1.99714286]]
-19988.703873392686 -2612.398717582956 -3657.6117376971906
Accuracy of model on training set is: 99.04761904761905%
Accuracy of model on test set is: 97.77777777777777% 

Training dataset size = 70.0% and testing dataset size = 30.0%
Prior distribution = [0.3 0.5 0.2]
[[5.00857143 3.42857143 1.48       0.25142857]
 [5.89142857 2.77714286 4.21428571 1.31428571]
 [6.62857143 2.94       5.59142857 2.00571429]]
-16200.722320815657 -2553.9515742276467 -3057.6681326850207
Accuracy of model on training set is: 99.04761904761905%
Accuracy of model on test set is: 97.77777777777777% 

Training dataset size = 50.0% and testing dataset size = 50.0%
Prior distribution = [0.33333333 0.33333333 0.33333333]
[[4.964 3.4   1.424 0.232]
 [5.976 2.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: RuntimeWarning: divide by zero encountered in log


# **Inference**

1. When dataset distribution is 50% and priors are all equal we get train and test accuracies almost same.

2. When train set is 30%  and priors are all equal we get better accuracy on train set at the cost of accuracy on the test set which is lower. 

3. When train set is smaller and but prior probabilities are different we have poor performance on test set (general trend).

4. When train set 70% and priors are different we get very good estimation on test set because of the small numbers on test case.

5. So optimal situation is to split the data in 50% and prior probabilities are same.